In [1]:
cd /home/ronghang/workspace/cmn/

/yy2/ronghang/workspace/cmn


In [2]:
import os
import json
import numpy as np

In [3]:
ANN_FILE_TRAIN = './exp-cub200/data/train_bboxes.json'
ANN_FILE_VAL = './exp-cub200/data/val_bboxes.json'
ANN_FILE_TEST = './exp-cub200/data/test_bboxes.json'

IMAGE_DIR = './exp-cub200/cub200-dataset/images/'
PROPOSAL_DIR = './exp-cub200/data/rpn_proposals/'

In [4]:
# imdb_format:
#   a list of images
# each image:
#   a dict
#   { "im_path": "...",
#     "regions": [[[x1, y1, x2, y2], sentence], ...],
#     "proposals": [[x1, y1, x2, y2], ...],
#     "misc": {"dataset": "coco", ...} }

def build_cub200_imdb(ann_file, image_dir, proposal_dir):
    with open(ann_file) as f:
        raw_anns = json.load(f)

    imdb = [{"im_path": os.path.join(image_dir, d['path']),
             "regions": _convert_grounded_phrases(d['grounded_phrases']),
             "proposals": np.load(os.path.join(proposal_dir, d['path'].replace('.jpg', '.npy')))}
            for d in raw_anns.values()]

    return imdb

def _convert_cub_bbox(bbox_x1x2y1y2):
    x1, x2, y1, y2 = bbox_x1x2y1y2
    return [x1, y1, x2, y2]

def _convert_grounded_phrases(grounded_phrases):
    converted = []
    for bbox, text_ann in grounded_phrases:
        _, tokens = text_ann
        bbox = _convert_cub_bbox(bbox)
        phrase = ' '.join(tokens)
        converted.append([bbox, phrase])
    return converted

In [5]:
imdb_train = build_cub200_imdb(ANN_FILE_TRAIN, IMAGE_DIR, PROPOSAL_DIR)
imdb_val = build_cub200_imdb(ANN_FILE_VAL, IMAGE_DIR, PROPOSAL_DIR)
imdb_test = build_cub200_imdb(ANN_FILE_TEST, IMAGE_DIR, PROPOSAL_DIR)

In [6]:
os.makedirs('./exp-cub200/data/imdb', exist_ok=True)
np.save('./exp-cub200/data/imdb/imdb_train.npy', imdb_train)
np.save('./exp-cub200/data/imdb/imdb_val.npy', imdb_val)
np.save('./exp-cub200/data/imdb/imdb_test.npy', imdb_test)

imdb_trainval = imdb_train+imdb_val
np.save('./exp-cub200/data/imdb/imdb_trainval.npy', imdb_trainval)

imdb_all = imdb_train+imdb_val+imdb_test
np.save('./exp-cub200/data/imdb/imdb_all.npy', imdb_all)